In [1]:
# Importing packages

#import pandas as pd
#import numpy as np
from collections import defaultdict
import math


In [2]:
def parse_file(filename):
    f = open(filename, 'r')
    raw = f.readlines()
    sentences = []
    s = []
    #s.append(( "START", "--s--"))
    for line in raw:
        if line.strip():
            token, tag , label = line.strip().split('\t')
            token = token.lower();
            s.append((token, tag ))
        else:
            s.append(( "END", "--e--"))
            sentences.append(s)
            s = []
            s.append(( "START", "--s--" ))

    return sentences

# Downloading Corpuses for traing the markove model

In [3]:
#Corpuses used for training and testing hosted on my git repo
# data taken from:  Universal Dependencies (UD)
#Corpus name: EWT English Web Treebank LDC2012T13

# blog email reviews social web


# Over 250k tokens (16.5k sentences)
#Testing data equal to around 10% of the training data


url = "https://raw.githubusercontent.com/DawoudTormos/NLP_POS_Corpuses/main/train_pos.conllu"# traing data with their pos
!wget --no-cache --backups=1 {url}


url = "https://raw.githubusercontent.com/DawoudTormos/NLP_POS_Corpuses/main/test_pos.conllu"#data1 for testing
!wget --no-cache --backups=1 {url}
url = "https://raw.githubusercontent.com/DawoudTormos/NLP_POS_Corpuses/main/test_pos_.conllu"#pos result for comparison and seeing the accuraccy
!wget --no-cache --backups=1 {url}

url = "https://raw.githubusercontent.com/DawoudTormos/NLP_POS_Corpuses/main/test_pos2.conllu"#data2 for testing
!wget --no-cache --backups=1 {url}
url = "https://raw.githubusercontent.com/DawoudTormos/NLP_POS_Corpuses/main/test_pos2_.conllu"#pos result for comparison and seeing the accuraccy
!wget --no-cache --backups=1 {url}



--2024-05-20 06:34:06--  https://raw.githubusercontent.com/DawoudTormos/NLP_POS_Corpuses/main/train_pos.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2963265 (2.8M) [text/plain]
Failed to rename train_pos.conllu to train_pos.conllu.1: (2) No such file or directory
Saving to: ‘train_pos.conllu’

train_pos.conllu    100%[===================>]   2.83M  --.-KB/s    in 0.05s   

2024-05-20 06:34:07 (51.7 MB/s) - ‘train_pos.conllu’ saved [2963265/2963265]

--2024-05-20 06:34:07--  https://raw.githubusercontent.com/DawoudTormos/NLP_POS_Corpuses/main/test_pos.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.

# Implementation note:

- For python 3.6 and beyond, dictionaries retain the insertion order.
- Their hash-based lookup makes them suitable for rapid value retrival. (As the hashed key value is retreive through a binary search (bisection). Therefore  O(log(n))
   

In [ ]:
# loading the training corpus
with open("train_pos.conllu", 'r') as f:
    training_corpus = f.readlines()


print(f"10 items of the training corpus list")
print(training_corpus[0:10])


10 items of the training corpus list
['Al\tPROPN\tNNP\n', '-\tPUNCT\tHYPH\n', 'Zaman\tPROPN\tNNP\n', ':\tPUNCT\t:\n', 'American\tADJ\tJJ\n', 'forces\tNOUN\tNNS\n', 'killed\tVERB\tVBD\n', 'Shaikh\tPROPN\tNNP\n', 'Abdullah\tPROPN\tNNP\n', 'al\tPROPN\tNNP\n']


In [ ]:
# load in the test corpus
with open("test_pos.conllu", 'r') as f:
    testing_corpus = f.readlines()

print("A sample of the test corpus")
print(testing_corpus[0:10])

A sample of the test corpus
['What\t0\t0\n', 'if\t0\t0\n', 'Google\t0\t0\n', 'Morphed\t0\t0\n', 'Into\t0\t0\n', 'GoogleOS\t0\t0\n', '?\t0\t0\n', '\n', 'What\t0\t0\n', 'if\t0\t0\n']


In [4]:
#Gettig the training corpuse to the Data type needed
#list of list of tuples
parsed_train_pos = parse_file("train_pos.conllu")



print(parsed_train_pos[0:5])



[[('al', 'PROPN'), ('-', 'PUNCT'), ('zaman', 'PROPN'), (':', 'PUNCT'), ('american', 'ADJ'), ('forces', 'NOUN'), ('killed', 'VERB'), ('shaikh', 'PROPN'), ('abdullah', 'PROPN'), ('al', 'PROPN'), ('-', 'PUNCT'), ('ani', 'PROPN'), (',', 'PUNCT'), ('the', 'DET'), ('preacher', 'NOUN'), ('at', 'ADP'), ('the', 'DET'), ('mosque', 'NOUN'), ('in', 'ADP'), ('the', 'DET'), ('town', 'NOUN'), ('of', 'ADP'), ('qaim', 'PROPN'), (',', 'PUNCT'), ('near', 'ADP'), ('the', 'DET'), ('syrian', 'ADJ'), ('border', 'NOUN'), ('.', 'PUNCT'), ('END', '--e--')], [('START', '--s--'), ('[', 'PUNCT'), ('this', 'DET'), ('killing', 'NOUN'), ('of', 'ADP'), ('a', 'DET'), ('respected', 'ADJ'), ('cleric', 'NOUN'), ('will', 'AUX'), ('be', 'AUX'), ('causing', 'VERB'), ('us', 'PRON'), ('trouble', 'NOUN'), ('for', 'ADP'), ('years', 'NOUN'), ('to', 'PART'), ('come', 'VERB'), ('.', 'PUNCT'), (']', 'PUNCT'), ('END', '--e--')], [('START', '--s--'), ('dpa', 'PROPN'), (':', 'PUNCT'), ('iraqi', 'ADJ'), ('authorities', 'NOUN'), ('announ

# Creating a vocabulary dictionary of all tokens present in the training corpus

In [ ]:
#Create a vocabulary set of all the used words in the training corpuse
vocab_s=set()

u = 0
uu = 0

#preparing the training corpuse
for s in parsed_train_pos :
    for w in range(len(s)):
      #print(w)
      vocab_s.add(s[w][0])
      if s[w][0]=="START" and w != 0:# checking if some "START" comes not in the start of a sentence
        u+=1
      if s[w][0]=="END" and w != len(s)-1:# checking if some "END" comes not in the end of a sentence
        uu+=1

print(u , uu)# result of checking "START" and "END" # if 0 , 0 then everything is good

print(len(vocab_s) , "unique tokens")
print(vocab_s)#Tokens of the set of unique tokens

0 0
17062 unique tokens
{'hours', 'o.k.', 'covered', 'lisp', 'teco', 'preparations', 'eat', 'attraction', 'lynda', 'plate', 'admits', 'gastric', 'desperate', 'devastating', "committee's", 'disasters', 'errors', '115', 'horton', 'mexicana', 'soviets', 'pretending', 'snacks', 'balance', '360', 'veronica', "meiring's", 'painless', 'gracie', 'stores', 'brendan', 'waitresses', 'lie', 'militias', 'difficulty', 'finance', 'associate', '08:27:46', '713-646-3393', 'lazy', 'rudest', 'shiite', 'wavy', 'racism', 'packard', 'inexperienced', 'mates', 'folks', 'originate', 'towel', 'notebook.url', 'gansu', 'references', 'areas', '703.729.2710', 'system', 'displeases', 'evidence', 'chlorine', 'contributions', 'offerings', 'transfer', 'contest', 'fiction', 'patrick', 'nicki', 'smallest', 'warrant', 'googol', 'gobbled', 'sap', 'staying', 'progressive', 'corporate', '03:33', '240', 'shave', 'suez', 'invoked', 'lng', '532-3836', 'dancing', 'ay', 'interested', 'telugu', 'tons', 'antisocialism', 'ends', '11

In [ ]:
# vocab dictionary where the key is the token and the value is the index ( Unique int)
vocab_dict = {}


for i, word in enumerate(sorted(vocab_s)):
    vocab_dict[word] = i



Testing with some words

In [ ]:
print(vocab_dict['#'])
print(vocab_dict['dog'])
print(vocab_dict['land'])
print(vocab_dict['see'])
print(vocab_dict['eat'])
print(vocab_dict['mosque'])

print("Vocab betwen 1650 and 1700")
cnt = 0


cnt = 0
for k,v in vocab_dict.items():
    if cnt > 1650:
      print(f"{k}:{v}")
    cnt += 1
    if cnt > 1700:
        break


print(vocab_dict["START"])
len(vocab_dict)

24
5301
9128
13754
5558
10374
Vocab betwen 1650 and 1700
alt.animals.lion:1651
alt.animals.rights.promotion:1652
alt.animals.tiger:1653
alt.consumers:1654
alta:1655
altar:1656
alter:1657
alterations:1658
altered:1659
altering:1660
alternate:1661
alternates:1662
alternating:1663
alternative:1664
alternatives:1665
although:1666
altitude:1667
alto:1668
altogether:1669
aluminum:1670
alumni:1671
always:1672
am:1673
amalgam:1674
amanda:1675
amaze:1676
amazed:1677
amazes:1678
amazing:1679
amazingly:1680
ambassador:1681
ambiance:1682
ambitions:1683
ambitious:1684
ambitiously:1685
ambulance:1686
ambulances:1687
amelia:1688
amen:1689
amend:1690
amended:1691
amendment:1692
amendments:1693
amendmnets:1694
amends:1695
america:1696
america's:1697
american:1698
americans:1699
americas:1700
1164


17062

# Preparing the data and counting needed before creating the matrices of probabilities

This part will beounting each transition and emission instance and put the values in dictionaries

In [ ]:

def create_dictionaries(parsed_train_pos, vocab_dict):


    # defaultdict for ease of use
    emission_instances_values = defaultdict(int)
    transition_instances_values = defaultdict(int)
    tag_values = defaultdict(int)

    # Starting at the start token
    prev_tag = '--s--'

    # Index for tracking
    i = 0

    # Going through each word and collecting data
    for  s in parsed_train_pos:
        for word_tag in s:

            i += 1

            # after each 20,000 word
            if i % 20000 == 0:
                print(f"word count = {i}")


            token, tag = word_tag[0]  , word_tag[1]
            #get_word_tag(word_tag, vocab_dict)

            # counting each transition instance
            transition_instances_values[(prev_tag, tag)] += 1

            # counting each emission instance
            emission_instances_values[(tag, token)] += 1


            tag_values[tag] += 1


            prev_tag = tag



    return emission_instances_values, transition_instances_values, tag_values

In [ ]:
emission_instances_values, transition_instances_values, tag_values = create_dictionaries(parsed_train_pos, vocab_dict)

word count = 20000
word count = 40000
word count = 60000
word count = 80000
word count = 100000
word count = 120000
word count = 140000
word count = 160000
word count = 180000
word count = 200000
word count = 220000


Making a list of all POS tags

In [ ]:
# get all the POS states
states = sorted(tag_values.items())
tags = sorted(tag_values.keys())
print(f"Number of POS tags (number of 'states'): {len(states)}")
print("View these POS tags (states)")
print(states)
print(tags)


Number of POS tags (number of 'states'): 20
View these POS tags (states)
[('--e--', 12544), ('--s--', 12543), ('ADJ', 13155), ('ADP', 17797), ('ADV', 10103), ('AUX', 12805), ('CCONJ', 6688), ('DET', 16300), ('INTJ', 695), ('NOUN', 34801), ('NUM', 4125), ('PART', 5708), ('PRON', 18660), ('PROPN', 12618), ('PUNCT', 23479), ('SCONJ', 3822), ('SYM', 720), ('VERB', 22602), ('X', 355), ('_', 2559)]
['--e--', '--s--', 'ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X', '_']


In [ ]:
print("transition examples: ")
for ex in list(transition_instances_values.items())[:3]:
    print(ex)
print()

print("emission examples: ")
for ex in list(emission_instances_values.items())[200:203]:
    print (ex)
print()

print("ambiguous word example: ")
for tup,cnt in emission_instances_values.items():
    if tup[1] == 'back': print (tup, cnt)

transition examples: 
(('--s--', 'PROPN'), 1560)
(('PROPN', 'PUNCT'), 3176)
(('PUNCT', 'PROPN'), 1343)

emission examples: 
(('NOUN', 'willingness'), 2)
(('ADJ', 'main'), 30)
(('NOUN', 'difference'), 20)

ambiguous word example: 
('ADV', 'back') 176
('VERB', 'back') 4
('ADP', 'back') 17
('ADJ', 'back') 5
('NOUN', 'back') 33
('X', 'back') 1


# Calculating Emissions probabilities from the collected values (Emissions Matrix)

In [ ]:
#del emission_probabilities
emission_probabilities = defaultdict(float) # initializing Emissions Matrix
x= 0
for tag in tags:
    total_emissions = 0
    for token in vocab_s:
       total_emissions += emission_instances_values[(tag, token)]
    for token in vocab_s:
        emission_probabilities[(tag, token)] = emission_instances_values[(tag, token)] / total_emissions

        #tests:
        if tag == "ADJ" and emission_probabilities[(tag, token)] > 0:
          #print(emission_probabilities[(tag, token)]*100 , tag , token)
          x+=emission_probabilities[(tag, token)]


print()
print(emission_probabilities[('--e--' , 'END')])
print(emission_probabilities[('--s--' , 'START')])
print(x)
#tests end


1.0
1.0
0.9999999999999766


# Calculating Transitions probabilities from the collected values (Transition Matrix)

In [ ]:
#del emission_probabilities
transition_probabilities = defaultdict(float) # initializing Transitions Matrix


print(   "training some expected values to see how correct the code was:")


for prev_tag in tags:
    total_transitions = 0
    for next_tag in tags:
       total_transitions += transition_instances_values[(prev_tag, next_tag)]
    for next_tag in tags:
        transition_probabilities[(prev_tag, next_tag)] = transition_instances_values[(prev_tag, next_tag)] / total_transitions

        #tests:
        if prev_tag == "--e--":
          print(transition_probabilities[(prev_tag, next_tag)] *100,prev_tag , next_tag )
        if prev_tag == "--s--":
          print(transition_probabilities[(prev_tag, next_tag)]*100 ,prev_tag , next_tag )
        if prev_tag == "ADV":
          print(transition_probabilities[(prev_tag, next_tag)]*100 ,prev_tag , next_tag )
        if prev_tag == "ADJ":
          print(transition_probabilities[(prev_tag, next_tag)]*100 ,prev_tag , next_tag )

print(transition_probabilities[('--s--' , 'NOUN')]*100 ,prev_tag , next_tag)
print(transition_probabilities[('--s--' , 'ADV')]*100 ,prev_tag , next_tag)
print(transition_probabilities[('--e--' , 'ADV')]*100 ,prev_tag , next_tag)
print(transition_probabilities[('--e--' , 'NOUN')]*100 ,prev_tag , next_tag)
#tests end

training some expected values to see how correct the code was:
0.0 --e-- --e--
100.0 --e-- --s--
0.0 --e-- ADJ
0.0 --e-- ADP
0.0 --e-- ADV
0.0 --e-- AUX
0.0 --e-- CCONJ
0.0 --e-- DET
0.0 --e-- INTJ
0.0 --e-- NOUN
0.0 --e-- NUM
0.0 --e-- PART
0.0 --e-- PRON
0.0 --e-- PROPN
0.0 --e-- PUNCT
0.0 --e-- SCONJ
0.0 --e-- SYM
0.0 --e-- VERB
0.0 --e-- X
0.0 --e-- _
0.0 --s-- --e--
0.0 --s-- --s--
4.081632653061225 --s-- ADJ
4.368622448979592 --s-- ADP
7.509566326530612 --s-- ADV
2.5510204081632653 --s-- AUX
2.3198341836734695 --s-- CCONJ
10.060586734693878 --s-- DET
3.2286352040816326 --s-- INTJ
6.194196428571429 --s-- NOUN
3.9142219387755106 --s-- NUM
0.47034438775510207 --s-- PART
25.247130102040817 --s-- PRON
12.436224489795919 --s-- PROPN
3.4040178571428568 --s-- PUNCT
3.5634566326530615 --s-- SCONJ
0.7732780612244898 --s-- SYM
5.970982142857143 --s-- VERB
0.007971938775510204 --s-- X
3.8982780612244894 --s-- _
0.3800836183960471 ADJ --e--
0.0 ADJ --s--
5.419992398327632 ADJ ADJ
7.6472824021

# Ambiguity examples that can be solved with hidden markove model

In [ ]:
for tup,val in emission_probabilities.items():
    if tup[1] == 'back': print (tup, val)
print("\n------\n")
for tup,val in emission_probabilities.items():
    if tup[1] == 'can': print (tup, val)
print("\n------\n")
for tup,val in emission_probabilities.items():
    if tup[1] == 'done': print (tup, val)

('--e--', 'back') 0.0
('--s--', 'back') 0.0
('ADJ', 'back') 0.0003800836183960471
('ADP', 'back') 0.0009552171714333876
('ADV', 'back') 0.017420568148074828
('AUX', 'back') 0.0
('CCONJ', 'back') 0.0
('DET', 'back') 0.0
('INTJ', 'back') 0.0
('NOUN', 'back') 0.0009482486135455878
('NUM', 'back') 0.0
('PART', 'back') 0.0
('PRON', 'back') 0.0
('PROPN', 'back') 0.0
('PUNCT', 'back') 0.0
('SCONJ', 'back') 0.0
('SYM', 'back') 0.0
('VERB', 'back') 0.00017697548889478808
('X', 'back') 0.0028169014084507044
('_', 'back') 0.0

------

('--e--', 'can') 0.0
('--s--', 'can') 0.0
('ADJ', 'can') 0.0
('ADP', 'can') 0.0
('ADV', 'can') 0.0
('AUX', 'can') 0.049199531433033974
('CCONJ', 'can') 0.0
('DET', 'can') 0.0
('INTJ', 'can') 0.0
('NOUN', 'can') 0.0001436740323553921
('NUM', 'can') 0.0
('PART', 'can') 0.0
('PRON', 'can') 0.0
('PROPN', 'can') 7.925186241876684e-05
('PUNCT', 'can') 0.0
('SCONJ', 'can') 0.0
('SYM', 'can') 0.0
('VERB', 'can') 0.0
('X', 'can') 0.0
('_', 'can') 0.0

------

('--e--', 'done

# Viterbi Algorithm
First part dealing with each sentence alone

In [ ]:
sent = ["START" ,"the" , "cat" , "ate" , "the" , "cake" , "END"]
sent = ["START" ,"the" , "back" , "of" , "the" , "school" , "END"]

def Viterbi_Sent(sent ,transition_probabilities ,emission_probabilities  ):

      result = ["--s--"]

      prev_token = sent[0]
      prev_tag= "--s--"



      path = defaultdict(float)
      #print(n_step)



      maxv = 0
      maxt = tuple()
      for i in range(len(sent) - 2):
        maxt = tuple()
        i+=1
        for curr_tag in tags:


            if prev_tag == "UNK":
                x = emission_probabilities[curr_tag , sent[i]] #* transition_probabilities[prev_tag , curr_tag]
            else:
                x = emission_probabilities[curr_tag , sent[i]] * transition_probabilities[prev_tag , curr_tag]
            if x != 0:


              for next_tag in tags:

                  if sent[i+1] in vocab_dict:
                      xx = transition_probabilities[ curr_tag , next_tag]
                  else:
                      xx = emission_probabilities[next_tag , sent[i+1]] * transition_probabilities[ curr_tag , next_tag]

                  #print(   emission_probabilities[next_tag , sent[i+1]]  ,  transition_probabilities[ curr_tag , next_tag] )


                  if xx != 0:
                    v = x  * xx
                  else:
                    v = x * 0.0000015
                    #print(v ,  curr_tag , next_tag , sent[i])    # uncomment this to viterbi in action


                  if v > maxv:
                    maxv = v
                    maxt = (curr_tag , next_tag)



                  '''else:
                    print('xx' ,sent[i], emission_probabilities[next_tag , sent[i+1]] , transition_probabilities[ curr_tag , next_tag])
            else:
              print('x' ,sent[i], emission_probabilities[curr_tag , sent[i]] , transition_probabilities[prev_tag , curr_tag])'''


        try:
          prev_tag =   maxt[0]
        except IndexError:
          prev_tag="X"

        result.append(prev_tag)
        maxv = 0


      result.append("--e--")
      return result


print(Viterbi_Sent(sent ,transition_probabilities ,emission_probabilities  ))


['--s--', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', '--e--']


# Testing the model for Accuracy

Loading testing Corpus 1

In [ ]:
parsed_testing_corpus = parse_file("test_pos.conllu")
parsed_testing_corpus[0].insert(0,('START' ,0 ,0))

In [ ]:
"into" in vocab_dict

True

Getting pos Tags using Viterbi Algorithm

In [ ]:
%%time
result =[]

for s in parsed_testing_corpus:
    temp=[]
    temp2=[]

    for w in s:
        temp.append(w[0])
    #print(temp)
    temp_result = Viterbi_Sent( temp ,transition_probabilities ,emission_probabilities  )
    #print(temp_result)
    for ii in range(len(s)):
        temp2.append((temp[ii] , temp_result[ii]))

    result.append(temp2)

#result

CPU times: user 813 ms, sys: 16.2 ms, total: 829 ms
Wall time: 837 ms


In [ ]:
parsed_testing_corpus_trueResult = parse_file("test_pos_.conllu")
parsed_testing_corpus_trueResult[0].insert(0,('START' ,0 ,0))
parsed_testing_corpus_trueResult[1]

[('START', '--s--'),
 ('what', 'PRON'),
 ('if', 'SCONJ'),
 ('google', 'PROPN'),
 ('expanded', 'VERB'),
 ('on', 'ADP'),
 ('its', 'PRON'),
 ('search', 'NOUN'),
 ('-', 'PUNCT'),
 ('engine', 'NOUN'),
 ('(', 'PUNCT'),
 ('and', 'CCONJ'),
 ('now', 'ADV'),
 ('e-mail', 'NOUN'),
 (')', 'PUNCT'),
 ('wares', 'NOUN'),
 ('into', 'ADP'),
 ('a', 'DET'),
 ('full', 'ADV'),
 ('-', 'PUNCT'),
 ('fledged', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('?', 'PUNCT'),
 ('END', '--e--')]

Measuring the model's Accuraccy

In [ ]:
total=0
corr = 0

for s in range(len(parsed_testing_corpus_trueResult)):
    for w in range(len(parsed_testing_corpus_trueResult[s])):
      total += 1
      if parsed_testing_corpus_trueResult[s][w][1] == result[s][w][1]:
        corr += 1

accuraccy = corr / total
print(int(accuraccy*100000)/1000 , "%")

86.374 %


In [ ]:
parsed_testing_corpus = parse_file("test_pos2.conllu")
parsed_testing_corpus[0].insert(0,('START' ,0 ,0))

In [ ]:
%%time
result =[]

for s in parsed_testing_corpus:
    temp=[]
    temp2=[]

    for w in s:
        temp.append(w[0])
    #print(temp)
    temp_result = Viterbi_Sent( temp ,transition_probabilities ,emission_probabilities  )
    #print(temp_result)
    for ii in range(len(s)):
        temp2.append((temp[ii] , temp_result[ii]))

    result.append(temp2)

#result

CPU times: user 1.42 s, sys: 2.32 ms, total: 1.43 s
Wall time: 1.49 s


In [ ]:
parsed_testing_corpus_trueResult = parse_file("test_pos2_.conllu")
parsed_testing_corpus_trueResult[0].insert(0,('START' ,0 ,0))
parsed_testing_corpus_trueResult[0]

[('START', 0, 0),
 ('from', 'ADP'),
 ('the', 'DET'),
 ('ap', 'PROPN'),
 ('comes', 'VERB'),
 ('this', 'DET'),
 ('story', 'NOUN'),
 (':', 'PUNCT'),
 ('END', '--e--')]

In [ ]:
total=0
corr = 0

for s in range(len(parsed_testing_corpus_trueResult)):
    for w in range(len(parsed_testing_corpus_trueResult[s])):
      total += 1
      if parsed_testing_corpus_trueResult[s][w][1] == result[s][w][1]:
        corr += 1

accuraccy = corr / total
print(int(accuraccy*100000)/1000 , "%")

86.403 %
